In [1]:
import os
import pandas as pd
import numpy as np

import gc
import pickle

from tqdm.notebook import tqdm_notebook as tqdm

import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import roc_auc_score

C:\Users\user\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")
C:\Users\user\Anaconda3\lib\site-packages\dask\config.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\user\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\user\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyy

# 데이터 불러오기

In [2]:
train = pd.read_csv('C:/Users/user/Downloads/data/가공train.csv')
train

,winner,game_id,time,player,species,event,event_contents
0,1,4886,0.38,0,T,Ability,(1022) - BuildRefinery; Target: DestructibleSi...
1,0,36244,9.04,0,P,GetControlGroup,NaN
2,1,25171,6.38,1,T,Camera,"at (49.12109375, 120.6015625)"
3,0,15241,1.48,1,Z,Camera,"at (26.5, 126.0078125)"
4,1,27626,5.25,0,Z,Camera,"at (75.84765625, 103.2734375)"
...,...,...,...,...,...,...,...
670913,0,22800,1.57,1,T,Right Click,"Location: (43.740478515625, 123.315185546875, ..."
670914,0,7584,8.20,0,T,Camera,"at (111.06640625, 30.609375)"
670915,0,35224,8.57,1,T,Camera,"at (39.203125, 32.48046875)"
670916,0,26802,9.58,1,P,Ability,(15E0) - TrainProbe


In [3]:
y = train['winner']
X = train.copy()
del X['winner']

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)

In [10]:
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536734 entries, 143094 to 448218
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   game_id         536734 non-null  int64  
 1   time            536734 non-null  float64
 2   player          536734 non-null  int64  
 3   species         536734 non-null  object 
 4   event           536734 non-null  object 
 5   event_contents  452416 non-null  object 
 6   winner          536734 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 32.8+ MB


In [12]:
train.isnull().sum()

game_id               0
time                  0
player                0
species               0
event                 0
event_contents    84318
winner                0
dtype: int64

# 데이터 전처리

1. 타입 변환


- int : 정수형 자료형 기본값 => int32, int64

- int8 : 2^8개수 만큼 표현 가능 (-128~127)
- unit8 : 양수만 표현 가능, 2^8개수 만큼 표현 가능 (0~255)

In [13]:
# 타입 변환 => 용량 감소
train['winner'] = train['winner'].astype(np.uint8)

for df in [train, test]:
    df['game_id'] = df['game_id'].astype(np.uint16)
    df['time'] = df['time'].astype(np.float64)
    df['player'] = df['player'].astype(np.uint8)
    df['species'] = df['species'].map({'P':0, 'T':1, 'Z':2}).astype(np.uint8)
    df['event'] = df['event'].map({'Ability':0, 'AddToControlGroup':1, 'Camera':2, 'ControlGroup':3, 'GetControlGroup':4, 'Right Click':5, 'Selection':6, 'SetControlGroup':7})

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536734 entries, 143094 to 448218
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   game_id         536734 non-null  uint16 
 1   time            536734 non-null  float64
 2   player          536734 non-null  uint8  
 3   species         536734 non-null  uint8  
 4   event           536734 non-null  int64  
 5   event_contents  452416 non-null  object 
 6   winner          536734 non-null  uint8  
dtypes: float64(1), int64(1), object(1), uint16(1), uint8(3)
memory usage: 18.9+ MB


2. 시간 관련 파생변수 생성


- 시간을 같은 단위로 맞추기
- game_id별 시간 quantile 뽑아내기

In [14]:
train['time'].describe()

count    536734.000000
mean          4.273876
std           2.710200
min           0.000000
25%           2.120000
50%           4.140000
75%           6.360000
max          10.580000
Name: time, dtype: float64

In [15]:
# game_id별 time의 quantile 컬럼을 추가하여 후에 여러가지 시점으로 볼 때 용이하고자 했습니다.

# 아래 단위로 변환 => 초시점으로
for df in [train, test]:
    df['time'] = (np.trunc(df['time'])*60 + (df['time'] - np.trunc(df['time'])) * 100).astype(int)

    
# game_id별 time의 quantile 컬럼을 추가하여 후에 여러가지 시점으로 볼 때 용이하고자 했습니다.
train_quantile = train[['game_id', 'time']].groupby('game_id').quantile([0.25, 0.5, 0.75, 1]).unstack()
train_quantile.columns = ['qt25', 'qt50', 'qt75', 'qt100']
train_quantile = train_quantile.reset_index()
train = pd.merge(train, train_quantile.astype(int), how='left', on='game_id')

test_quantile = test[['game_id', 'time']].groupby('game_id').quantile([0.25, 0.5, 0.75, 1]).unstack()
test_quantile.columns = ['qt25', 'qt50', 'qt75', 'qt100']
test_quantile = test_quantile.reset_index()
test = pd.merge(test, test_quantile.astype(int), how='left', on='game_id')

In [16]:
train

,game_id,time,player,species,event,event_contents,winner,qt25,qt50,qt75,qt100
0,2371,542,1,2,2,"at (87.85546875, 69.04296875)",1,192,262,415,574
1,38849,262,1,2,2,"at (38.55078125, 95.5234375)",0,102,224,267,415
2,18722,411,0,2,2,"at (150.01953125, 106.22265625)",0,169,305,353,411
3,15232,142,0,1,2,"at (150.2734375, 29.59765625)",0,124,285,369,578
4,36641,266,0,1,2,"at (142.1015625, 30.91015625)",1,176,256,307,380
...,...,...,...,...,...,...,...,...,...,...,...
536729,3722,140,0,0,2,"at (152.0, 123.0859375)",0,95,156,241,301
536730,29373,102,0,1,5,"Target: Refinery [03A40002]; Location: (150.5,...",0,180,294,416,575
536731,3476,105,1,0,5,"Location: (22.07568359375, 135.544189453125, 4...",1,111,244,381,505
536732,30486,230,1,1,4,NaN,0,165,237,413,493


3. event가 2인 데이터


- event가 2인 데이터의 game_id와 player를 활용해 새로운 변수 생성
- 위 변수를 통해 groupby 별 evnet_contents 뽑기
- 결측치 nan 처리
- game_id 별 player 별 event_contents 쪼개서 df 생성

In [17]:
train_dd = train.loc[train['event']==2, ['game_id', 'player', 'event_contents']]
train_dd['game_p'] = train_dd['game_id'].astype(str) + '_' + train_dd['player'].astype(str)
train_dd

,game_id,player,event_contents,game_p
0,2371,1,"at (87.85546875, 69.04296875)",2371_1
1,38849,1,"at (38.55078125, 95.5234375)",38849_1
2,18722,0,"at (150.01953125, 106.22265625)",18722_0
3,15232,0,"at (150.2734375, 29.59765625)",15232_0
4,36641,0,"at (142.1015625, 30.91015625)",36641_0
...,...,...,...,...
536721,38605,1,"at (27.41015625, 114.81640625)",38605_1
536726,11604,1,"at (150.8984375, 37.72265625)",11604_1
536727,12242,0,"at (111.8984375, 159.83984375)",12242_0
536729,3722,0,"at (152.0, 123.0859375)",3722_0


In [18]:
train_dd = train_dd.groupby('game_p')['event_contents'].first()
train_dd = train_dd.reset_index()
train_dd['game_id'] = train_dd['game_p'].apply(lambda x: x.split('_')[0])
train_dd['player'] = train_dd['game_p'].apply(lambda x: x.split('_')[1])

train_dd = train_dd.groupby(['game_id', 'player'])['event_contents'].first().unstack()
train_dd.index = train_dd.index.astype(int)
train_dd = train_dd.sort_index()
train_dd.columns = [0, 1]
train_dd

,0,1
game_id,,
0,"at (137.46484375, 34.7421875)","at (24.3515625, 139.48046875)"
1,"at (135.6015625, 159.3125)","at (140.0, 112.02734375)"
2,"at (147.234375, 41.88671875)","at (49.1796875, 104.390625)"
3,"at (116.26953125, 46.3046875)","at (26.49609375, 120.515625)"
4,"at (49.546875, 119.36328125)","at (138.1953125, 47.25)"
...,...,...
38867,"at (26.62890625, 128.984375)","at (143.59375, 40.2578125)"
38868,"at (38.46484375, 22.75)","at (73.83984375, 28.05859375)"
38869,"at (149.9453125, 28.4609375)","at (113.91796875, 20.23828125)"


In [19]:
# 결측치는 nan으로 처리
train_dd = train_dd.fillna(np.nan)

In [20]:
train_dd['p0_x'] = train_dd[0].apply(lambda x: str(x)[4:].split(',')[0])
train_dd['p0_y'] = train_dd[0].apply(lambda x: str(x).split(', ')[-1][:-1])

train_dd['p1_x'] = train_dd[1].apply(lambda x: str(x)[4:].split(',')[0])
train_dd['p1_y'] = train_dd[1].apply(lambda x: str(x).split(', ')[-1][:-1])
train_dd

,0,1,p0_x,p0_y,p1_x,p1_y
game_id,,,,,,
0,"at (137.46484375, 34.7421875)","at (24.3515625, 139.48046875)",137.46484375,34.7421875,24.3515625,139.48046875
1,"at (135.6015625, 159.3125)","at (140.0, 112.02734375)",135.6015625,159.3125,140.0,112.02734375
2,"at (147.234375, 41.88671875)","at (49.1796875, 104.390625)",147.234375,41.88671875,49.1796875,104.390625
3,"at (116.26953125, 46.3046875)","at (26.49609375, 120.515625)",116.26953125,46.3046875,26.49609375,120.515625
4,"at (49.546875, 119.36328125)","at (138.1953125, 47.25)",49.546875,119.36328125,138.1953125,47.25
...,...,...,...,...,...,...
38867,"at (26.62890625, 128.984375)","at (143.59375, 40.2578125)",26.62890625,128.984375,143.59375,40.2578125
38868,"at (38.46484375, 22.75)","at (73.83984375, 28.05859375)",38.46484375,22.75,73.83984375,28.05859375
38869,"at (149.9453125, 28.4609375)","at (113.91796875, 20.23828125)",149.9453125,28.4609375,113.91796875,20.23828125


In [21]:
train_dd.loc[train_dd['p0_x']=='', 'p0_y'] = -1
train_dd.loc[train_dd['p0_x']=='', 'p0_x'] = -1
train_dd.loc[train_dd['p1_y']=='na', 'p1_x'] = -1
train_dd.loc[train_dd['p1_y']=='na', 'p1_y'] = -1
train_dd = train_dd[['p0_x', 'p0_y', 'p1_x', 'p1_y']].astype(float).reset_index(drop=True)
train_dd

,p0_x,p0_y,p1_x,p1_y
0,137.464844,34.742188,24.351562,139.480469
1,135.601562,159.312500,140.000000,112.027344
2,147.234375,41.886719,49.179688,104.390625
3,116.269531,46.304688,26.496094,120.515625
4,49.546875,119.363281,138.195312,47.250000
...,...,...,...,...
37415,26.628906,128.984375,143.593750,40.257812
37416,38.464844,22.750000,73.839844,28.058594
37417,149.945312,28.460938,113.917969,20.238281
37418,149.281250,133.160156,32.222656,36.296875


In [24]:
test_dd = test.loc[test['event']==2, ['game_id', 'player', 'event_contents']]
test_dd['game_p'] = test_dd['game_id'].astype(str) + '_' + test_dd['player'].astype(str)
test_dd = test_dd.groupby('game_p')['event_contents'].first()
test_dd = test_dd.reset_index()
test_dd['game_id'] = test_dd['game_p'].apply(lambda x: x.split('_')[0])
test_dd['player'] = test_dd['game_p'].apply(lambda x: x.split('_')[1])

test_dd = test_dd.groupby(['game_id', 'player'])['event_contents'].first().unstack()
test_dd.index = test_dd.index.astype(int)
test_dd = test_dd.sort_index()
test_dd.columns = [0, 1]

test_dd = test_dd.fillna(np.nan)

test_dd['p0_x'] = test_dd[0].apply(lambda x: str(x)[4:].split(',')[0])
test_dd['p0_y'] = test_dd[0].apply(lambda x: str(x).split(', ')[-1][:-1])

test_dd['p1_x'] = test_dd[1].apply(lambda x: str(x)[4:].split(',')[0])
test_dd['p1_y'] = test_dd[1].apply(lambda x: str(x).split(', ')[-1][:-1])

test_dd.loc[test_dd['p0_x']=='', 'p0_y'] = -1
test_dd.loc[test_dd['p0_x']=='', 'p0_x'] = -1
test_dd.loc[test_dd['p1_y']=='na', 'p1_x'] = -1
test_dd.loc[test_dd['p1_y']=='na', 'p1_y'] = -1
test_dd = test_dd[['p0_x', 'p0_y', 'p1_x', 'p1_y']].astype(float).reset_index(drop=True)
test_dd

,p0_x,p0_y,p1_x,p1_y
0,136.300781,36.894531,24.351562,138.175781
1,45.316406,56.890625,-1.000000,-1.000000
2,147.234375,41.886719,45.906250,104.707031
3,-1.000000,-1.000000,103.847656,65.480469
4,56.136719,115.480469,-1.000000,-1.000000
...,...,...,...,...
28728,32.003906,142.125000,140.882812,35.394531
28729,-1.000000,-1.000000,135.902344,141.789062
28730,40.449219,96.257812,-1.000000,-1.000000
28731,-1.000000,-1.000000,34.503906,20.375000


# 변수 선택 및 모델 구축

1. Transform


- game_id : unique() 처리
- winner : game_id 별 평균 winner
- time : game_id 별 최대 time
- player : game_id 별 player_0/player_1 count 개수
- species : game_id 별 species_0/species_1 count 개수
- event : game_id와 player별 event 각각의 count 개수

In [26]:
def Transform(train_df, is_train=True):
    transform_train_df = pd.DataFrame()

    # game_id, winner, time
    transform_train_df['game_id'] = train_df['game_id'].unique()
    transform_train_df.index = train_df['game_id'].unique()

    if is_train:
        transform_train_df['winner'] = train_df.groupby('game_id')['winner'].mean()
    transform_train_df['time'] = train_df.groupby('game_id')['time'].max()
    
    # player
    temp = train_df.groupby('game_id')['player'].value_counts().unstack()
    temp.columns = ['p0_count', 'p1_count']
    transform_train_df = pd.concat([transform_train_df, temp], 1)

    temp = train_df.groupby(['game_id', 'player'])['species'].first().unstack()
    temp.columns = ['p0_species', 'p1_species']
    transform_train_df = pd.concat([transform_train_df, temp], 1)

    temp = train_df.groupby(['game_id', 'player'])['event'].value_counts().unstack().unstack().fillna(0.0)
    temp.columns = ['p0_event0', 'p1_event0', 'p0_event1', 'p1_event1', 'p0_event2', 'p1_event2', 'p0_event3', 'p1_event3', 
                    'p0_event4', 'p1_event4', 'p0_event5', 'p1_event5', 'p0_event6', 'p1_event6', 'p0_event7', 'p1_event7']
    transform_train_df = pd.concat([transform_train_df, temp], 1)

    # event contents
    train_event_contents0 = train_df.loc[train_df['event']==0, ['game_id', 'player', 'event_contents']]
    train_event_contents0['event_contents'] = train_event_contents0['event_contents'].apply(lambda x: str(x).split(' - ')[0])
    train_event_contents0['event_contents'] = 'p' + train_event_contents0['player'].astype(str) + '_' + train_event_contents0['event_contents']
    train_event_contents0.drop(columns='player', inplace=True)
    train_event_contents0 = train_event_contents0.groupby('game_id')['event_contents'].value_counts().unstack().fillna(0.0)

    train_event_contents2 = train_df.loc[train_df['event']==2, ['game_id', 'player', 'event_contents']]
    train_event_contents2['event_contents'] = train_event_contents2['event_contents'].apply(lambda x: x[:2])
    train_event_contents2['event_contents'] = 'p' + train_event_contents2['player'].astype(str) + '_' + train_event_contents2['event_contents']
    train_event_contents2.drop(columns='player', inplace=True)
    train_event_contents2 = train_event_contents2.groupby('game_id')['event_contents'].value_counts().unstack().fillna(0.0)

    train_event_contents = pd.concat([train_event_contents0, train_event_contents2], 1)
    del train_event_contents0, train_event_contents2

    train_event_contents6 = train_df.loc[train_df['event']==6, ['game_id', 'player', 'event_contents']]
    train_event_contents6['event_contents'] = train_event_contents6['event_contents'].apply(lambda x: str(x).split()[0][2:])
    train_event_contents6['event_contents'] = 'p' + train_event_contents6['player'].astype(str) + '_' + train_event_contents6['event_contents']
    train_event_contents6.drop(columns='player', inplace=True)
    train_event_contents6 = train_event_contents6.groupby('game_id')['event_contents'].value_counts().unstack().fillna(0.0)

    train_event_contents = pd.concat([train_event_contents, train_event_contents6], 1)
    del train_event_contents6

    train_event_contents5 = train_df.loc[train_df['event']==5, ['game_id', 'player', 'event_contents']]
    train_event_contents5['event_contents'] = train_event_contents5['event_contents'].apply(lambda x: 'Location' if 'Target' not in x else 'Target').value_counts()
    train_event_contents5['event_contents'] = 'p' + train_event_contents5['player'].astype(str) + '_' + train_event_contents5['event_contents']
    train_event_contents5.drop(columns='player', inplace=True)
    train_event_contents5 = train_event_contents5.groupby('game_id')['event_contents'].value_counts().unstack().fillna(0.0)

    train_event_contents = pd.concat([train_event_contents, train_event_contents5], 1)
    del train_event_contents5
    
    transform_train_df =pd.concat([transform_train_df, train_event_contents], 1).reset_index(drop=True)

    for col in [col for col in transform_train_df.columns if col not in ['game_id', 'winner', 'time', 'p0_species', 'p1_species']]:
        transform_train_df[f'isnan_{col}'] = (transform_train_df[col]>0).astype(int)
        if len(transform_train_df[f'isnan_{col}'].value_counts())==1:
            transform_train_df.drop(columns=f'isnan_{col}', inplace=True)
    return transform_train_df

2. reverse_copy


- 0과 1 바꾸기

In [27]:
def reverse_copy(train_df, is_train=True):
    if is_train:
        train_reverse = train_df.iloc[:, :3].copy()
        train_reverse['winner'] = train_reverse['winner'].map({1:0, 0:1})
    else:
        train_reverse = train_df.iloc[:, :2].copy()

    temp_reverse = train_df[[col for col in train_df.columns if 'p0' in col]]
    temp_reverse.columns = [col.replace('p0', 'p1') for col in temp_reverse.columns]
    train_reverse = pd.concat([train_reverse, temp_reverse], 1)

    temp_reverse = train_df[[col for col in train_df.columns if 'p1' in col]]
    temp_reverse.columns = [col.replace('p1', 'p0') for col in temp_reverse.columns]
    train_reverse2 = pd.concat([train_reverse, temp_reverse], 1)

    train_df = pd.concat([train_df, train_reverse2], sort=False, ignore_index=True).fillna(0.0)
    return train_df

In [28]:
# 1. 3가지의 time을 기준으로 데이터셋을 생성하였고 lgb모델을 이용했습니다.
# 2. 후에 weight average로 앙상블을 진행했습니다.
train4 = {}
test4 = {}

####################################################################################

train2 = Transform(train[train['qt50']<=train['time']], is_train=True)
test2 = Transform(test[test['qt50']<=test['time']], is_train=False)

train2 = pd.concat([train2, train_dd], 1)
test2 = pd.concat([test2, test_dd], 1)

train3 = reverse_copy(train2, is_train=True)
test3 = reverse_copy(test2, is_train=False)


train4['50'] = train3
test4['50'] = test3

####################################################################################

train2 = Transform(train[train['qt25']<=train['time']], is_train=True)
test2 = Transform(test[test['qt25']<=test['time']], is_train=False)

train2 = pd.concat([train2, train_dd], 1)
test2 = pd.concat([test2, test_dd], 1)

train3 = reverse_copy(train2, is_train=True)
test3 = reverse_copy(test2, is_train=False)

train4['25'] = train3
test4['25'] = test3

####################################################################################

train2 = Transform(train, is_train=True)
test2 = Transform(test, is_train=False)

train2 = pd.concat([train2, train_dd], 1)
test2 = pd.concat([test2, test_dd], 1)

train3 = reverse_copy(train2, is_train=True)
test3 = reverse_copy(test2, is_train=False)

train4['x'] = train3
test4['x'] = test3

In [29]:
Transform(train[train['qt50']<=train['time']], is_train=True)

,game_id,winner,time,p0_count,p1_count,p0_species,p1_species,p0_event0,p1_event0,p0_event1,...,isnan_p1_VikingAssault,isnan_p1_VoidRay,isnan_p1_WarpGate,isnan_p1_WarpPrism,isnan_p1_WarpPrismPhasing,isnan_p1_Weapon,isnan_p1_XelNagaTower,isnan_p1_Zealot,isnan_p1_Zergling,isnan_p1_ZerglingBurrowed
0,0,1,379,2.0,5.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,598,5.0,4.0,0.0,1.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,595,3.0,4.0,0.0,2.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,582,7.0,5.0,1.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,291,6.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38497,38867,1,464,NaN,2.0,NaN,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
38498,38868,0,580,5.0,2.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
38499,38869,0,494,7.0,4.0,0.0,2.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
38500,38870,1,332,1.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
train4['50']

,game_id,winner,time,p0_count,p1_count,p0_species,p1_species,p0_event0,p1_event0,p0_event1,...,isnan_p0_AutoTurret,isnan_p0_BarracksReactor,isnan_p0_Broodling,isnan_p0_ChangelingMarine,isnan_p0_ChangelingZerglingWings,isnan_p0_QueenBurrowed,isnan_p0_RichMineralField,isnan_p0_RoachBurrowed,isnan_p0_XelNagaTower,isnan_p0_ZerglingBurrowed
0,0,1,379,2.0,5.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,598,5.0,4.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,595,3.0,4.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,582,7.0,5.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,291,6.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76999,38867,0,464,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77000,38868,1,580,2.0,5.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77001,38869,1,494,4.0,7.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77002,38870,0,332,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
train4['25']

,game_id,winner,time,p0_count,p1_count,p0_species,p1_species,p0_event0,p1_event0,p0_event1,...,isnan_p0_AiurLightBridgeNE8,isnan_p0_AutoTurret,isnan_p0_Broodling,isnan_p0_ChangelingMarine,isnan_p0_ChangelingZerglingWings,isnan_p0_QueenBurrowed,isnan_p0_RichMineralField,isnan_p0_RoachBurrowed,isnan_p0_XelNagaTower,isnan_p0_ZerglingBurrowed
0,0,1,379,4.0,6.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,598,8.0,5.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,595,3.0,7.0,0.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,582,10.0,8.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,291,8.0,2.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76999,38867,0,464,2.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77000,38868,1,580,4.0,6.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77001,38869,1,494,6.0,10.0,2.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77002,38870,0,332,6.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
train4['x']

,game_id,winner,time,p0_count,p1_count,p0_species,p1_species,p0_event0,p1_event0,p0_event1,...,isnan_p0_AiurLightBridgeNE8,isnan_p0_AutoTurret,isnan_p0_Broodling,isnan_p0_ChangelingMarine,isnan_p0_ChangelingZerglingWings,isnan_p0_QueenBurrowed,isnan_p0_RichMineralField,isnan_p0_RoachBurrowed,isnan_p0_XelNagaTower,isnan_p0_ZerglingBurrowed
0,0,1,379,5.0,8.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,598,12.0,6.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,595,5.0,9.0,0.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,582,14.0,10.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,291,10.0,4.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76999,38867,0,464,2.0,2.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77000,38868,1,580,6.0,7.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77001,38869,1,494,7.0,15.0,2.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77002,38870,0,332,8.0,2.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 모델 학습 및 검증

In [33]:
def make_predictions(train_df, test_df, features, n_folds=5, seed=42):
    skf = StratifiedKFold(n_splits=n_folds, random_state=seed, shuffle=True)
    lgb_params = {
        'objective':'binary',
        'boosting_type':'gbdt',
        'metric':'auc',
        'n_jobs':-1,
        'learning_rate':0.01,
        'num_leaves': 500,
        'max_depth':-1,
        'min_child_weight':5,
        'colsample_bytree': 0.3,
        'subsample':0.7,
        'gamma':0,
        'reg_lambda':0.1,
        'reg_alpha':0.1,
        'verbose':-1,
        'seed': seed
    }

    oof_np = np.zeros(len(train_df))
    pred_np = np.zeros(len(test_df))

    for trn_idx, val_idx in skf.split(train_df, train_df[TARGET]):

        tt = lgb.Dataset(train_df.loc[trn_idx, features], label=train_df.loc[trn_idx, TARGET])
        vv = lgb.Dataset(train_df.loc[val_idx, features], label=train_df.loc[val_idx, TARGET])

        clf = lgb.train(lgb_params, tt, 20000, valid_sets=[tt, vv], early_stopping_rounds=300, verbose_eval=500)
        
        oof_np[val_idx] = clf.predict(train_df.loc[val_idx, features])
        pred_np += clf.predict(test_df[features]) / N_FOLDS
        print()
    return oof_np, pred_np

In [34]:
N_FOLDS = 5
TARGET = 'winner'
SEED = 42

oof = {}
pred = {}

for table in train4.keys():

    remove_columns = ['game_id', TARGET]
    features_columns = [col for col in train4[table].columns if col not in remove_columns]
    print(f'feature갯수 : {len(features_columns)}개')

    oof[table], pred[table] = make_predictions(train4[table], test4[table], features=features_columns, n_folds=N_FOLDS, seed=SEED)

feature갯수 : 1667개
[LightGBM] [Warning] Unknown parameter: gamma
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[74]	training's auc: 0.854673	valid_1's auc: 0.552028


KeyError: "['isnan_p0_Changeling', 'isnan_p0_(1BA1)', 'p1_(16E9)', 'isnan_p0_(182B)', 'isnan_p0_(1500)', 'isnan_p1_(1441)', 'p0_ChangelingZerglingWings', 'isnan_p0_TechLab', 'isnan_p1_(8E0)', 'p0_StarportReactor', 'p1_OverseerCocoon', 'p1_ChangelingZealot', 'p1_(1D83)', 'isnan_p1_(102F)', 'isnan_p1_OverseerCocoon', 'isnan_p0_(2180)', 'p1_(1443)', 'p1_(1A63)', 'isnan_p0_(1700)', 'p1_(1204)', 'p1_(8E0)', 'p0_(17E1)', 'p1_SwarmHost', 'p1_(17E1)', 'p1_AiurLightBridgeNE8', 'p1_RoachBurrowed', 'isnan_p0_(14E9)', 'isnan_p1_(14E9)', 'isnan_p1_(1500)', 'isnan_p0_(F00)', 'p1_(2140)', 'isnan_p1_(15C2)', 'isnan_p0_(1029)', 'p0_(1800)', 'isnan_p1_(1440)', 'p1_(880)', 'p0_(1D81)', 'p1_NydusWorm', 'p1_Weapon', 'isnan_p0_ZerglingBurrowed', 'isnan_p1_(1029)', 'isnan_p1_StarportReactor', 'isnan_p1_(17E1)', 'p0_(55E0)', 'p0_(1B21)', 'isnan_p1_(42A1)', 'p1_DestructibleIce2x4Vertical', 'isnan_p1_(1A63)', 'p0_(1029)', 'isnan_p0_StarportReactor', 'isnan_p0_(1440)', 'p0_(15C2)', 'p0_MissileTurret', 'p1_(1BA1)

# 결과 및 결언

In [ ]:
def pred_transform(pred, thres=38872):
    pred = np.array(pred)
    return pred[:thres]/2 + (1-pred[thres:])/2

sub['winner'] = pred_transform(pred['25], 16787)*.4 + pred_transform(pred['x'], 16787)*.4
sub.to_csv('submission.csv', index=False)